Reprocessing overall_scores 
Jira Ticket: https://sagebionetworks.jira.com/browse/AG-220

Tasks:
- Subtract flyneuropath scores from logsdon scores
- include ensembl_gene_id

In [12]:
import synapseclient
import pandas as pd
import numpy as np

syn = synapseclient.Synapse()
syn.login(silent=True)

interesting_columns = ['ENSG', 'GeneName', 'Logsdon', 'GeneticsScore', 'OmicsScore', 'LiteratureScore', 'FlyNeuroPathScore']

overall_scores = 'syn25575156'
overall_scores = syn.tableQuery(f'SELECT * FROM {overall_scores}').asDataFrame()

overall_scores


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.5.0) is available. Your version (2.4.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.5.0 release notes

https://python-docs.synapse.org/build/html/news.html



,GeneName,Logsdon,GeneticsScore,OmicsScore,LiteratureScore,FlyNeuroPathScore,SM_Druggability_bucket,safety_bucket,feasibility_bucket,ABability_bucket,...,safety_bucket_definition,feasibility_bucket_definition,ABability_bucket_definition,new_modality_bucket_definition,tissue_engagement_bucket_definition,ENSG,isScored_genetics,isScored_omics,isScored_lit,isScored_neuropath
144869_154,SCRIB,7.404221,1.618243,1.988317,1.79766153846154,2,3,4,2,3,...,More than two of: high off target gene express...,High target qualification but medium feasibili...,Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",ENSG00000180900,Y,Y,Y,Y
144870_154,GRIN2B,7.165803,2.141011,1.686699,1.33809230769231,2,1,1,2,1,...,"Clinical data, evidence of tolerable safety pr...",High target qualification but medium feasibili...,Secreted protein. Highly accessible to antibod...,Not specified suitable for degradation/inhibit...,"The target gene is “tissue enriched, “group en...",ENSG00000273079,Y,Y,Y,Y
144871_154,GABRB3,6.878784,1.439669,1.641453,1.79766153846154,2,1,1,2,3,...,"Clinical data, evidence of tolerable safety pr...",High target qualification but medium feasibili...,Cell membrane-bound proteins. Highly accessibl...,Not specified suitable for degradation/inhibit...,"The target gene is “tissue enriched, “group en...",ENSG00000166206,Y,Y,Y,Y
144872_154,SPR,6.720777,1.723829,1.976886,1.02006153846154,2,3,4,3,5,...,More than two of: high off target gene express...,High target qualification but low feasibility ...,Protein located in the cytosol. Not practicall...,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",ENSG00000116096,Y,Y,Y,Y
144873_154,DEK,6.688744,1.499666,1.996955,1.19212307692308,2,12,4,3,6,...,More than two of: high off target gene express...,High target qualification but low feasibility ...,Protein located in intracellular compartment.,Not specified suitable for degradation/inhibit...,"The target gene is not """"tissue enriched""""/""""t...",ENSG00000124795,Y,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173717_184,NaN,0.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ENSG00000280399,N,Y,N,N
173718_184,NaN,0.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ENSG00000280222,N,Y,N,N
173719_184,NaN,0.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ENSG00000285883,N,Y,N,N
173720_184,NaN,0.000000,0.000000,0.000000,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ENSG00000275120,N,Y,N,N


AG-235 in the cell below: https://sagebionetworks.jira.com/browse/AG-235

In [13]:
is_scored = ['isScored_genetics', 'isScored_omics', 'isScored_lit', 'isScored_neuropath']

for left, right in zip(interesting_columns[3:], is_scored):
    overall_scores[left] = np.where(overall_scores[right] == 'Y', overall_scores[left], np.nan)

In [14]:
# subtract the FlyNeuroPathScore from Logsdon
overall_scores['Logsdon'] = overall_scores['Logsdon'] - overall_scores['FlyNeuroPathScore']
overall_scores.drop(columns=['FlyNeuroPathScore'], inplace=True)
interesting_columns.remove('FlyNeuroPathScore')
overall_scores['Logsdon'].value_counts(dropna=False) #17650 / 10706

NaN         10706
0.000000       58
0.107866       28
0.663056       10
0.079845        9
            ...  
0.613474        1
2.186156        1
1.231696        1
1.127419        1
0.258339        1
Name: Logsdon, Length: 17651, dtype: int64

In [13]:
overall_scores = overall_scores[interesting_columns]

In [14]:
overall_scores.to_json('../output/overall_scores_235.json', orient='records', indent=2)